<center>
<img src="https://habrastorage.org/webt/ia/m9/zk/iam9zkyzqebnf_okxipihkgjwnw.jpeg">
    
## [mlcourse.ai](mlcourse.ai) – Open Machine Learning Course 

Author: [Yury Kashnitskiy](https://yorko.github.io). 
Translated and edited by [Maxim Keremet](https://www.linkedin.com/in/maximkeremet/), [Artem Trunov](https://www.linkedin.com/in/datamove/), and [Aditya Soni](https://www.linkedin.com/in/aditya-soni-0505a9124/). This material is subject to the terms and conditions of the [Creative Commons CC BY-NC-SA 4.0](https://creativecommons.org/licenses/by-nc-sa/4.0/) license. Free use is permitted for any non-commercial purpose.

# <center> Solution #2. <br> Visual data analysis in Python: EDA of US flights <br> (using Pandas, Matplotlib & Seaborn & Plotly)

<img src='../../img/flight.png' width=50%>


* **Your task** is to write code and perform computations in the cells below

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

import plotly.graph_objs as go
import plotly.express as px

# enable static images of your plot embedded in the notebook
%matplotlib inline 

sns.set() #apply the default default seaborn theme, scaling, and color palette

# Graphics in retina format are more sharp and legible
%config InlineBackend.figure_format = 'retina'

* Download the data [archive](http://www.rdatasciencecases.org/Data/Airline/2008.csv.bz2) (Archived ~ 114 Mb, unzipped - ~ 690 Mb). No need to unzip - pandas can unbzip on the fly.
* Place it in the "../../data" folder, or change the path below according to your location.
* The dataset has information about carriers and flights between US airports during the year 2008. 
* Column description is available [here](http://www.transtats.bts.gov/Fields.asp?Table_ID=236). Visit this site to find ex. meaning of flight cancellation codes.

**Reading data into memory and creating a Pandas _DataFrame_ object**

(This may take a while, be patient)

We are not going to read in the whole dataset. In order to reduce memory footprint, we instead load only needed columns and cast them suitable data types.

In [2]:
dtype = {'DayOfWeek': np.uint8, 'DayofMonth': np.uint8, 'Month': np.uint8 , 'Cancelled': np.uint8, 
         'Year': np.uint16, 'FlightNum': np.uint16 , 'Distance': np.uint16, 
         'UniqueCarrier': str, 'CancellationCode': str, 'Origin': str, 'Dest': str,
         'ArrDelay': np.float16, 'DepDelay': np.float16, 'CarrierDelay': np.float16,
         'WeatherDelay': np.float16, 'NASDelay': np.float16, 'SecurityDelay': np.float16,
         'LateAircraftDelay': np.float16, 'DepTime': np.float16}

In [ ]:
%%time
# change the path if needed
path = '../../data/2008.csv.bz2'
flights_df = pd.read_csv(path, usecols=dtype.keys(), dtype=dtype)

**Check the number of rows and columns and print column names.**

In [ ]:
print(flights_df.shape)
print(flights_df.columns)

**Print first 5 rows of the dataset.**

In [ ]:
flights_df.head()

**Transpose the frame to see all features at once.**

In [ ]:
flights_df.head().T

**Examine data types of all features and total dataframe size in memory.**

In [ ]:
flights_df.info()

**Get basic statistics of each feature.**

In [ ]:
flights_df.describe().T

**Count unique Carriers and plot their relative share of flights:**

In [ ]:
flights_df['UniqueCarrier'].nunique()

In [ ]:
flights_df.groupby('UniqueCarrier').size().plot(kind='bar');

**We can also _group by_ category/categories in order to calculate different aggregated statistics.**

**For example, finding top-3 flight codes, that have the largest total distance travelled in year 2008.**

In [ ]:
flights_df.groupby(['UniqueCarrier','FlightNum'])['Distance'].sum().sort_values(ascending=False).iloc[:3]

**Another way:**

In [ ]:
flights_df.groupby(['UniqueCarrier','FlightNum'])\
  .agg({'Distance': [np.mean, np.sum, 'count'],
        'Cancelled': np.sum})\
  .sort_values(('Distance', 'sum'), ascending=False)\
  .iloc[0:3]

**Number of flights by days of week and months:**

In [ ]:
pd.crosstab(flights_df.Month, flights_df.DayOfWeek)

**It can also be handy to color such tables in order to easily notice outliers:**

In [ ]:
plt.imshow(pd.crosstab(flights_df.Month, flights_df.DayOfWeek),
           cmap='seismic', interpolation='none');

**Flight distance histogram:**

In [ ]:
flights_df.hist('Distance', bins=20);

**Making a histogram of flight frequency by date.**

In [ ]:
flights_df['Date'] = pd.to_datetime(flights_df.rename(columns={'DayofMonth': 'Day'})[['Year', 'Month', 'Day']])

In [ ]:
num_flights_by_date = flights_df.groupby('Date').size()

In [ ]:
num_flights_by_date.plot();

**Do you see a weekly pattern above? And below?**

In [ ]:
num_flights_by_date.rolling(window=7).mean().plot();

**We'll need a new column in our dataset - departure hour, let's create it.**

As we see, `DepTime` is distributed from 1 to 2400 (it is given in the `hhmm` format, check the [column description](https://www.transtats.bts.gov/Fields.asp?Table_ID=236) again). We'll treat departure hour as `DepTime` // 100 (divide by 100 and apply the `floor` function). However, now we'll have both hour 0 and hour 24. Hour 24 sounds strange, we'll set it to be 0 instead (a typical imperfectness of real data, however, you can check that it affects only 521 rows, which is sort of not a big deal). So now values of a new column `DepHour` will be distributed from 0 to 23. There are some missing values, for now we won't fill in them, just ignore them. 

In [ ]:
flights_df['DepHour'] = flights_df['DepTime'] // 100
flights_df['DepHour'].replace(to_replace=24, value=0, inplace=True)

In [ ]:
flights_df['DepHour'].describe()

**1. Find top-10 carriers in terms of the number of completed flights (_UniqueCarrier_ column)?**

**Which of the listed below is _not_ in your top-10 list?**
- DL
- AA
- OO
- EV <span style="color:green"> **[+]** </span>

In [ ]:
# solution
top10_airlines = flights_df.loc[flights_df['Cancelled'] == 0,
                                'UniqueCarrier'].value_counts().head(10)
top10_airlines

**2. Plot distributions of flight cancellation reasons (_CancellationCode_).**

**What is the most frequent reason for flight cancellation? (Use this [link](https://www.transtats.bts.gov/Fields.asp?Table_ID=236) to translate codes into reasons)**
- carrier
- weather conditions <span style="color:green"> **[+]** </span>
- National Air System
- security reasons

****
*Reminder on Cancellatoin codes:*
```
A - Carrier 
B - Weather 
C - National Air System 
D - Security
```
****

In [ ]:
# solution
plt.figure(figsize=(10, 5))
sns.countplot(flights_df['CancellationCode'].sort_values());

In [ ]:
flights_df['CancellationCode'].mode()

**3. Which route is the most frequent, in terms of the number of flights?**

(Take a look at _'Origin'_ and _'Dest'_ features. Consider _A->B_ and _B->A_ directions as _different_ routes) 

 - New-York – Washington
 - San-Francisco – Los-Angeles <span style="color:green"> **[+]** </span>
 - San-Jose – Dallas
 - New-York – San-Francisco

In [ ]:
# solution
flights_df['Route'] = flights_df['Origin'] + '->' + flights_df['Dest']
flights_df['Route'].value_counts().head()

**4. Find top-5 delayed routes (count how many times they were delayed on departure). From all flights on these 5 routes, count all flights with weather conditions contributing to a delay.**

**Hint:** consider only positive delays


- 449 
- 539 
- 549 
- 668 <span style="color:green"> **[+]** </span>

In [ ]:
# solution
# find top5 routes with most delayed flights
top5_delayed = flights_df[flights_df['DepDelay'] > 0].groupby('Route')\
    .size().sort_values(ascending=False).head(5)

top5_delayed

In [ ]:
# reduce to only flights from top5 delayed routes
flights_top5_delays = flights_df[flights_df['Route'].isin(top5_delayed.index)]

In [ ]:
# now the answer
(flights_top5_delays['WeatherDelay'] > 0).sum()

**5. Examine the hourly distribution of departure times. For that, create a new series from DepTime, removing missing values.**

**Choose all correct statements:**
 - Flights are normally distributed within time interval [0-23] (Search for: Normal distribution, bell curve).
 - Flights are uniformly distributed within time interval [0-23].   
 - In the period from 0 am to 4 am there are considerably less flights than from 7 pm to 8 pm. <span style="color:green"> **[+]** </span>

In [ ]:
# solution
hour_group = flights_df['DepHour'].value_counts(sort=False)

plt.figure(figsize=(10, 5))
sns.barplot(x=hour_group.index, y=hour_group)
plt.xticks(rotation=90)
plt.show()

In [ ]:
# Checking the 3rd statements separately
round((flights_df['DepHour'] == 19).sum() / (flights_df['DepHour'] < 5).sum())

**6. Show how the number of flights changes through time (on the daily/weekly/monthly basis) and interpret the findings.**

**Choose all correct statements:**
- The number of flights during weekends is less than during weekdays (working days). <span style="color:green"> **[+]** </span>
- The lowest number of flights is on Sunday.
- There are less flights during winter than during summer. <span style="color:green"> **[+]** </span>

In [ ]:
# solution
num_flights_by_day_of_week = flights_df[flights_df['Cancelled'] == 0].groupby(
    'DayOfWeek').size()


plt.figure(figsize=(10, 5))
sns.barplot(x=num_flights_by_day_of_week.index, y=num_flights_by_day_of_week)

In [ ]:
num_flights_by_month = flights_df[flights_df['Cancelled'] == 0].groupby('Month').size()

plt.figure(figsize=(10, 5))
sns.barplot(x=num_flights_by_month.index, y=num_flights_by_month);

**7. Examine the distribution of cancellation reasons with time. Make a bar plot of cancellation reasons aggregated by months.**

**Choose all correct statements:**
- December has the highest rate of cancellations due to weather. <span style="color:green"> **[+]** </span>
- The highest rate of cancellations in September is due to Security reasons.
- April's top cancellation reason is carriers. <span style="color:green"> **[+]** </span>
- Flights cancellations due to National Air System are more frequent than those due to carriers.

****
*Reminder on Cancellatoin codes:*
```
A - Carrier 
B - Weather 
C - National Air System 
D - Security
```
****

In [ ]:
# create a month name list
import calendar

month_names = []
for month_idx in flights_df['Month'].unique():
    month_names.append((calendar.month_name[month_idx]))

cancel_df = (flights_df.groupby(['Month', 'CancellationCode'])
             .size()
             .reset_index()
             .rename(columns={0: 'count'}))

# create a barplot
plt.figure(figsize=(15, 5))

plot = sns.barplot(
    x=cancel_df['Month'], y=cancel_df['count'], hue=cancel_df['CancellationCode'])
plot.set_xticklabels(month_names, rotation=90)
plot.legend(loc='upper left');

**8. Which month has the greatest number of cancellations due to Carrier?** 
- May
- January
- September
- April <span style="color:green"> **[+]** </span>

In [ ]:
# solution
flights_df.loc[flights_df['CancellationCode'] == 'A', 'Month'].value_counts().head()

In [ ]:
cancel_df = (flights_df[flights_df['CancellationCode'] == 'A']
             .groupby(['Month'])['UniqueCarrier'].count()
             .reset_index()
             .rename(columns={'UniqueCarrier': 'count'}))

plt.figure(figsize=(15, 5))
plot = sns.barplot(
    x=cancel_df['Month'], y=cancel_df['count'])
plot.set_xticklabels(month_names, rotation=90);

**9. Identify the carrier with the greatest number of cancellations due to carrier in the corresponding month from the previous question.**

- 9E
- EV
- HA
- AA <span style="color:green"> **[+]** </span>

In [ ]:
# solution
flights_df.loc[(flights_df['CancellationCode'] == 'A') & (flights_df['Month'] == 4),
               'UniqueCarrier'].value_counts().head()

Or, as a nice plot (with much more efforts of course):

In [ ]:
import matplotlib.colors as colors
import random 

colors_list = list(colors._colors_full_map.values())
selected_colors = random.sample(colors_list,20)

cancelled_Carrier_April = (flights_df[(flights_df['CancellationCode'] == 'A') 
                                     & (flights_df['Month'] == 4)]
                .groupby(['Date', 'UniqueCarrier']).size().unstack())

# get the peak coordinates for annotation
max_value = cancelled_Carrier_April.max().max()
carrier_of_max_value = cancelled_Carrier_April.max().idxmax()
date_of_max_value = cancelled_Carrier_April[carrier_of_max_value].idxmax()
print("\n", "Most cancellated carrier:", carrier_of_max_value,
      "\n", "Number of cancellations:", max_value, 
      "\n", "Date:", date_of_max_value)

fig, ax = plt.subplots()

cancelled_Carrier_April.plot(color=selected_colors, figsize=(10,8), ax=ax)
ax.annotate(carrier_of_max_value, (str(date_of_max_value), max_value))
fig.autofmt_xdate()

**or**

In [ ]:
cancelled_Carrier_April = (flights_df[(flights_df['CancellationCode'] == 'A') 
                                     & (flights_df['Month'] == 4)]
                .groupby(['Date', 'UniqueCarrier']).size().reset_index()
                .rename(columns={0: 'count'}))

# get dates range
unique_dates = sorted(list(cancelled_Carrier_April['Date'].drop_duplicates()))
date_ticks = range(0, len(unique_dates), 4)

plt.figure(figsize=(15, 7))
g = sns.pointplot(x="Date", y="count", hue="UniqueCarrier", data=cancelled_Carrier_April)
g.set_xticks(date_ticks)
g.set_xticklabels([unique_dates[i].strftime('%d') for i in date_ticks])

plt.annotate(str(carrier_of_max_value), xy=(8, max_value), xytext=(8, max_value),
             bbox={'pad':4, 'edgecolor':'orange', 'facecolor': 
                   'orange', 'alpha':0.4});

**10. Examine median arrival and departure delays (in time) by carrier. Which carrier has the lowest median delay time for both arrivals and departures? Leave only non-negative values of delay times ('ArrDelay', 'DepDelay').
[Boxplots](https://seaborn.pydata.org/generated/seaborn.boxplot.html) can be helpful in this exercise, as well as it might be a good idea to remove outliers in order to build nice graphs. You can exclude delay time values higher than a corresponding .95 percentile.**

- EV
- OO
- AA
- AQ <span style="color:green"> **[+]** </span>

In [ ]:
# solution
for delay_type in ['ArrDelay', 'DepDelay']:
    sub_df = flights_df[(flights_df[delay_type] >= 0) &
                        (flights_df[delay_type] < 
                         flights_df[delay_type].quantile(.95))]
    plt.figure(figsize=(15, 7))
    sns.boxplot(x='UniqueCarrier', y=delay_type, data=sub_df)
    plt.show()